In [15]:
import requests
import os

# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = os.getenv('ODDS_API')

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h,spreads' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)
    # Check the usage quota
print('Remaining requests', odds_response.headers['x-requests-remaining'])
print('Used requests', odds_response.headers['x-requests-used'])

Remaining requests 380
Used requests 120


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, insert, Table, MetaData
from sqlalchemy.exc import SQLAlchemyError
import os
import getters


DATABASE_TYPE = os.getenv("DATABASE_TYPE")
DBAPI = os.getenv("DBAPI")
ENDPOINT = os.getenv("ENDPOINT")
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWD")
PORT = os.getenv("PORT")
DATABASE = os.getenv("DATABASE")


In [ ]:
def insert_user(name, email):
    connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"

    try:
        engine = create_engine(connection_string)

        with engine.connect() as connection:
            # Create an insert query
            metadata = MetaData()

            users_table = Table('users', metadata, autoload_with=engine)
            insert_query = insert(users_table).values(email=f"'{email}'", name=f"'{name}'")
            compiled = insert_query.compile()
            result = connection.execute(insert_query)
            connection.commit()
            
            user_id = result.inserted_primary_key[0]
            return user_id

    except SQLAlchemyError as e:
        print(f"Error interacting with the database: {e}")
        return None

In [ ]:
insert_user(name='Fred Eglin1', email='feglin701@gmail.com')

In [ ]:
def get_game_ids():
    connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"

    try:
        engine = create_engine(connection_string)
    except Exception as e:
        print(f"Error creating engine: {e}")

    sql_query = """
    select odds.game_id from odds;
    """

    game_ids_df = pd.read_sql(sql_query, con=engine)
    
    return game_ids_df

In [ ]:
print(get_game_ids())